In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import urllib
import re
import time
%matplotlib inline

In [ ]:
import nltk
import string
from nltk.tokenize import word_tokenize

import math

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [ ]:
tool_keywords1 = ['python', 'pytorch', 'sql', 'mxnet', 'mlflow', 'einstein', 'theano', 'pyspark', 'solr', 'mahout', 
 'cassandra', 'aws', 'powerpoint', 'spark', 'pig', 'sas', 'java', 'nosql', 'docker', 'salesforce', 'scala', 'r',
 'c', 'c++', 'net', 'tableau', 'pandas', 'scikitlearn', 'sklearn', 'matlab', 'scala', 'keras', 'tensorflow', 'clojure',
 'caffe', 'scipy', 'numpy', 'matplotlib', 'vba', 'spss', 'linux', 'azure', 'cloud', 'gcp', 'mongodb', 'mysql', 'oracle', 
 'redshift', 'snowflake', 'kafka', 'javascript', 'qlik', 'jupyter', 'perl', 'bigquery', 'unix', 'react',
 'scikit', 'powerbi', 's3', 'ec2', 'lambda', 'ssrs', 'kubernetes', 'hana', 'spacy', 'tf', 'django', 'sagemaker',
 'seaborn', 'mllib', 'github', 'git', 'elasticsearch', 'splunk', 'airflow', 'looker', 'rapidminer', 'birt', 'pentaho', 
 'jquery', 'nodejs', 'd3', 'plotly', 'bokeh', 'xgboost', 'rstudio', 'shiny', 'dash', 'h20', 'h2o', 'hadoop', 'mapreduce', 
 'hive', 'cognos', 'angular', 'nltk', 'flask', 'node', 'firebase', 'bigtable', 'rust', 'php', 'cntk', 'lightgbm', 
 'kubeflow', 'rpython', 'unixlinux', 'postgressql', 'postgresql', 'postgres', 'hbase', 'dask', 'ruby', 'julia', 'tensor']

# another set of keywords that are longer than one word.
tool_keywords2 = set(['amazon web services', 'google cloud', 'sql server'])

In [ ]:
# hard skills/knowledge required.
skill_keywords1 = set(['statistics', 'cleansing', 'chatbot', 'cleaning', 'blockchain', 'causality', 'correlation', 'bandit', 'anomaly', 'kpi',
 'dashboard', 'geospatial', 'ocr', 'econometrics', 'pca', 'gis', 'svm', 'svd', 'tuning', 'hyperparameter', 'hypothesis',
 'salesforcecom', 'segmentation', 'biostatistics', 'unsupervised', 'supervised', 'exploratory',
 'recommender', 'recommendations', 'research', 'sequencing', 'probability', 'reinforcement', 'graph', 'bioinformatics',
 'chi', 'knn', 'outlier', 'etl', 'normalization', 'classification', 'optimizing', 'prediction', 'forecasting',
 'clustering', 'cluster', 'optimization', 'visualization', 'nlp', 'c#',
 'regression', 'logistic', 'nn', 'cnn', 'glm',
 'rnn', 'lstm', 'gbm', 'boosting', 'recurrent', 'convolutional', 'bayesian',
 'bayes'])


# another set of keywords that are longer than one word.
skill_keywords2 = set(['random forest', 'natural language processing', 'machine learning', 'decision tree', 'deep learning', 'experimental design',
 'time series', 'nearest neighbors', 'neural network', 'support vector machine', 'computer vision', 'machine vision', 'dimensionality reduction', 
 'text analytics', 'power bi', 'a/b testing', 'ab testing', 'chat bot', 'data mining'])

In [ ]:
degree_dict = {'bs': 1, 'bachelor': 1, 'undergraduate': 1, 
               'master': 2, 'graduate': 2, 'mba': 2.5, 
               'phd': 3, 'ph.d': 3, 'ba': 1, 'ma': 2,
               'postdoctoral': 4, 'postdoc': 4, 'doctorate': 3}


degree_dict2 = {'advanced degree': 2, 'ms or': 2, 'ms degree': 2, '4 year degree': 1, 'bs/': 1, 'ba/': 1,
                '4-year degree': 1, 'b.s.': 1, 'm.s.': 2, 'm.s': 2, 'b.s': 1, 'phd/': 3, 'ph.d.': 3, 'ms/': 2,
                'm.s/': 2, 'm.s./': 2, 'msc/': 2, 'master/': 2, 'master\'s/': 2, 'bachelor\s/': 1}
degree_keywords2 = set(degree_dict2.keys())

In [ ]:
df = pd.read_csv('/Users/tingtingweng/Documents/summer_intern/linkedin_jobs_052620-062520.csv')
df.head()

,Title,Company,Location,Date,Seniority level,Employment type,Industry,Description,State,Code
0,Data Science Tester,SAIC,"Huntsville, Alabama, United States",2020-06-25,Entry level,Full-time,"Computer Hardware,Defense & Space,Information ...",DescriptionSAIC is seeking a Remote Data Scien...,Alabama,AL
1,ORSA- Sr. Data Scientist,SAIC,"Huntsville, Alabama, United States",2020-06-25,Associate,Full-time,"Computer Hardware,Defense & Space,Information ...",DescriptionJOB DESCRIPTIONSAIC has a current n...,Alabama,AL
2,Data Scientist,Cgi Technologies Corporation,"Birmingham, Alabama, United States",2020-06-24,Entry level,Full-time,"Information Technology and Services,Computer S...",Position Description Do you want to take your ...,Alabama,AL
3,Mid-Level Data Analyst,"Dynamis, Inc.","Prattville, Alabama, United States",2020-06-23,Entry level,Full-time,"Information Technology and Services,Defense & ...",Dynamis is seeking a Mid-level Data Analyst to...,Alabama,AL
4,Data Scientist,CGI,"Birmingham, Alabama, United States",2020-06-23,Entry level,Full-time,"Information Technology and Services,Computer S...",Job DescriptionJob Description Data ScientistP...,Alabama,AL


### Remove stopwords and punctuations

In [ ]:
from nltk.corpus import stopwords

stop = stopwords.words('english')

def cleanup(sentence):
    sentence = str(sentence)
    sentence = sentence.lower() # lower case
    sentence = re.sub(r'[?|!|.|,|)|(|\|/]',r' ',sentence) # replace these punctuation with space
    tokens = sentence.split()
    #remove stopwords
    for t in tokens:
        if t in stop:
            tokens.remove(t)
    out = []
    #for t in tokens:
    #    out.append(snow.stem(t))
    out = " ".join(tokens)
    out = re.sub(r'[\'|"|#]', r'', out) # remove these punctuation
    return out

In [ ]:
df['description_clean'] = df['Description'].apply(cleanup)

### Tokenizing and Parts of Speech (POS) Tagging

In [ ]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
from nltk.stem import PorterStemmer

pos_tag(tool_keywords1)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tingtingweng/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[('python', 'NN'),
 ('pytorch', 'NN'),
 ('sql', 'JJ'),
 ('mxnet', 'NN'),
 ('mlflow', 'NN'),
 ('einstein', 'NN'),
 ('theano', 'NN'),
 ('pyspark', 'NN'),
 ('solr', 'NN'),
 ('mahout', 'NN'),
 ('cassandra', 'NN'),
 ('aws', 'VBZ'),
 ('powerpoint', 'JJ'),
 ('spark', 'NN'),
 ('pig', 'NN'),
 ('sas', 'NN'),
 ('java', 'NN'),
 ('nosql', 'JJ'),
 ('docker', 'NN'),
 ('salesforce', 'NN'),
 ('scala', 'NN'),
 ('r', 'NN'),
 ('c', 'VBP'),
 ('c++', 'JJ'),
 ('net', 'JJ'),
 ('tableau', 'NN'),
 ('pandas', 'NN'),
 ('scikitlearn', 'NN'),
 ('sklearn', 'NN'),
 ('matlab', 'NN'),
 ('scala', 'NN'),
 ('keras', 'NNS'),
 ('tensorflow', 'JJ'),
 ('clojure', 'NN'),
 ('caffe', 'NN'),
 ('scipy', 'NN'),
 ('numpy', 'JJ'),
 ('matplotlib', 'NN'),
 ('vba', 'NN'),
 ('spss', 'NN'),
 ('linux', 'JJ'),
 ('azure', 'NN'),
 ('cloud', 'NN'),
 ('gcp', 'NN'),
 ('mongodb', 'NN'),
 ('mysql', 'NN'),
 ('oracle', 'NN'),
 ('redshift', 'NN'),
 ('snowflake', 'VBP'),
 ('kafka', 'NN'),
 ('javascript', 'NN'),
 ('qlik', 'NN'),
 ('jupyter', 'NN'),
 ('

In [ ]:
nltk.download('punkt')
ps = PorterStemmer()

# process the job description.
def prepare_job_desc(desc):
    # tokenize description.
    tokens = word_tokenize(desc)
        
    # Parts of speech (POS) tag tokens.
    token_tag = pos_tag(tokens)
    
    # Only include some of the POS tags.
    include_tags = ['VBN', 'VBD', 'JJ', 'JJS', 'JJR', 'CD', 'NN', 'NNS', 'NNP', 'NNPS']
    filtered_tokens = [tok for tok, tag in token_tag if tag in include_tags]
    
    # stem words.
    stemmed_tokens = [ps.stem(tok).lower() for tok in filtered_tokens]
    return set(stemmed_tokens)

df['description_word_set'] = df['description_clean'].map(prepare_job_desc)

# process the keywords
tool_keywords1_set = set([ps.stem(tok) for tok in tool_keywords1]) # stem the keywords (since the job description is also stemmed.)
tool_keywords1_dict = {ps.stem(tok):tok for tok in tool_keywords1} # use this dictionary to revert the stemmed words back to the original.

skill_keywords1_set = set([ps.stem(tok) for tok in skill_keywords1])
skill_keywords1_dict = {ps.stem(tok):tok for tok in skill_keywords1}

degree_keywords1_set = set([ps.stem(tok) for tok in degree_dict.keys()])
degree_keywords1_dict = {ps.stem(tok):tok for tok in degree_dict.keys()}

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tingtingweng/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tool_list = []
skill_list = []
degree_list = []

num_postings = len(df.index)
for i in range(num_postings):
    job_desc = df.iloc[i]['description_clean'].lower()
    job_desc_set = df.iloc[i]['description_word_set']
    
    # check if the keywords are in the job description. Look for exact match by token.
    tool_words = tool_keywords1_set.intersection(job_desc_set)
    skill_words = skill_keywords1_set.intersection(job_desc_set)
    degree_words = degree_keywords1_set.intersection(job_desc_set)
    
    # check if longer keywords (more than one word) are in the job description. Match by substring.
    j = 0
    for tool_keyword2 in tool_keywords2:
        # tool keywords.
        if tool_keyword2 in job_desc:
            tool_list.append(tool_keyword2)
            j += 1
    
    k = 0
    for skill_keyword2 in skill_keywords2:
        # skill keywords.
        if skill_keyword2 in job_desc:
            skill_list.append(skill_keyword2)
            k += 1
    
    # search for the minimum education.
    min_education_level = 999
    for degree_word in degree_words:
        level = degree_dict[degree_keywords1_dict[degree_word]]
        min_education_level = min(min_education_level, level)
    
    for degree_keyword2 in degree_keywords2:
        # longer keywords. Match by substring.
        if degree_keyword2 in job_desc:
            level = degree_dict2[degree_keyword2]
            min_education_level = min(min_education_level, level)
    
    # label the job descriptions without any tool keywords.
    if len(tool_words) == 0 and j == 0:
        tool_list.append('nothing specified')
    
    # label the job descriptions without any skill keywords.
    if len(skill_words) == 0 and k == 0:
        skill_list.append('nothing specified')
    
    # If none of the keywords were found, but the word degree is present, then assume it's a bachelors level.
    if min_education_level > 500:
        if 'degree' in job_desc:
            min_education_level = 1
    
    tool_list += list(tool_words)
    skill_list += list(skill_words)
    degree_list.append(min_education_level)

In [ ]:
# create the list of tools.
df_tool = pd.DataFrame(data={'cnt': tool_list})
df_tool = df_tool.replace(tool_keywords1_dict)

# group some of the categories together.
msk = df_tool['cnt'] == 'h20'
df_tool.loc[msk, 'cnt'] = 'h2o'

msk = df_tool['cnt'] == 'aws'
df_tool.loc[msk, 'cnt'] = 'amazon web services'

msk = df_tool['cnt'] == 'gcp'
df_tool.loc[msk, 'cnt'] = 'google cloud'

msk = df_tool['cnt'] == 'github'
df_tool.loc[msk, 'cnt'] = 'git'

msk = df_tool['cnt'] == 'postgressql'
df_tool.loc[msk, 'cnt'] = 'postgres'

msk = df_tool['cnt'] == 'tensor'
df_tool.loc[msk, 'cnt'] = 'tensorflow'

df_tool_top50 = df_tool['cnt'].value_counts().reset_index().rename(columns={'index': 'tool'}).iloc[:50]

In [ ]:
# visualize the tools.
layout = dict(
    title='Tools For Data Scientists',
    xaxis=dict(
        title='Tools and Programming Langragues',
        tickangle=60
    ),
    yaxis=dict(
        title='% of job postings',
        tickformat=',.0%'
    )
)

fig = go.Figure(layout=layout)
fig.add_trace(go.Bar(
    x=df_tool_top50['tool'],
    y=df_tool_top50['cnt']/num_postings
))

iplot(fig)

### Knowledge/Concepts

In [ ]:
# create the list of skills/knowledge.
df_skills = pd.DataFrame(data={'cnt': skill_list})
df_skills = df_skills.replace(skill_keywords1_dict)

# group some of the categories together.
msk = df_skills['cnt'] == 'nlp'
df_skills.loc[msk, 'cnt'] = 'natural language processing'

msk = df_skills['cnt'] == 'convolutional'
df_skills.loc[msk, 'cnt'] = 'convolutional neural network'

msk = df_skills['cnt'] == 'cnn'
df_skills.loc[msk, 'cnt'] = 'convolutional neural network'

msk = df_skills['cnt'] == 'recurrent'
df_skills.loc[msk, 'cnt'] = 'recurrent neural network'

msk = df_skills['cnt'] == 'rnn'
df_skills.loc[msk, 'cnt'] = 'recurrent neural network'

msk = df_skills['cnt'] == 'knn'
df_skills.loc[msk, 'cnt'] = 'nearest neighbors'

msk = df_skills['cnt'] == 'svm'
df_skills.loc[msk, 'cnt'] = 'support vector machine'

msk = df_skills['cnt'] == 'machine vision'
df_skills.loc[msk, 'cnt'] = 'computer vision'

msk = df_skills['cnt'] == 'ab testing'
df_skills.loc[msk, 'cnt'] = 'a/b testing'

df_skills_top50 = df_skills['cnt'].value_counts().reset_index().rename(columns={'index': 'skill'}).iloc[:50]


In [ ]:
# visualize the skills.
layout = dict(
    title='Knowledge/Skills For Data Scientists',
    xaxis=dict(
        title='Knowledge and Skills',
        tickangle=60
    ),
    yaxis=dict(
        title='% of job postings',
        tickformat=',.0%',
    )
)

fig = go.Figure(layout=layout)
fig.add_trace(go.Bar(
    x=df_skills_top50['skill'],
    y=df_skills_top50['cnt']/num_postings
))

iplot(fig)

### Minimum Education Required

In [ ]:
# create the list of degree.
df_degrees = pd.DataFrame(data={'cnt': degree_list})
df_degrees['degree_type'] = ''


msk = df_degrees['cnt'] == 1
df_degrees.loc[msk, 'degree_type'] = 'bachelors'

msk = df_degrees['cnt'] == 2
df_degrees.loc[msk, 'degree_type'] = 'masters'

msk = df_degrees['cnt'] == 3
df_degrees.loc[msk, 'degree_type'] = 'phd'

msk = df_degrees['cnt'] == 4
df_degrees.loc[msk, 'degree_type'] = 'postdoc'

msk = df_degrees['cnt'] == 2.5
df_degrees.loc[msk, 'degree_type'] = 'mba'

msk = df_degrees['cnt'] > 500
df_degrees.loc[msk, 'degree_type'] = 'not specified'


df_degree_cnt = df_degrees['degree_type'].value_counts().reset_index().rename(columns={'index': 'degree'}).iloc[:50]


In [ ]:
# visualize the degrees.
layout = dict(
    title='Minimum Education For Data Scientists',
    yaxis=dict(
        title='% of job postings',
        tickformat=',.0%',
    )
)

fig = go.Figure(layout=layout)
fig.add_trace(go.Bar(
    x=df_degree_cnt['degree'],
    y=df_degree_cnt['degree_type']/num_postings
))

iplot(fig)